# Vector search with LangChain (Azure AI Search)
このサンプルノートブックでは LangChain を使用して Azure AI Search の以下の検索クエリーを試すことができます。

- キーワード検索
- ベクトル検索
- ハイブリッド検索
- セマンティックハイブリッド検索



# 事前準備
この Python サンプルを実行するには、以下が必要です：
- Azure AI Search リソース。エンドポイントとクエリ API キーが必要です。
- Azure OpenAI Service にアクセスできる承認済み Azure サブスクリプション
- Azure OpenAI Service への `text-embedding-ada-002` Embeddings モデルのデプロイメント。このデモでは、API バージョン `2023-05-15` を使用しています。デプロイ名はモデルと同じ「`text-embedding-ada-002`」を使用しています。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI Embeddings モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。

## パッケージのインストール

<div class="alert alert-block alert-success">
<b>Note:</b> LangChain の pull request <a href="https://github.com/langchain-ai/langchain/pull/14789">#14789</a> がリリースに組み込まれるまでは、azure-search-documents==11.4.0b8 を使用する必要があります。</div>


In [ ]:
!pip uninstall -y azure-search-documents
!pip install azure-search-documents==11.4.0b8
#pip install azure-search-documents==11.4.0

In [ ]:
!pip install openai[datalib]==1.3.9
!pip install langchain==0.0.350

In [ ]:
import azure.search.documents
print("azure.search.documents", azure.search.documents.__version__)
import openai
print("openai", openai.__version__)
import langchain
print("langchain", langchain.__version__)

## 必要なライブラリと環境変数のインポート

In [ ]:
import os

os.environ["AZURESEARCH_FIELDS_CONTENT"]="content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"]="embedding"

from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

## 接続設定

In [ ]:
vector_store_address: str = "<Your search service endpoint>"
vector_store_password: str = "<Your search service query key>"

os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

model: str = "embedding" #自動構築時のデフォルト設定
index_name: str = "gptkbindex" #自動構築時のデフォルト設定


In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=model,
    openai_api_version="2023-05-15"
)

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="default"
)

# 1. キーワード検索
最もシンプルなキーワード検索のクエリーです。`ja.lucene` というスタンダードな日本語アナライザーに搭載されている辞書ベースのトークナイザーによって、これらのトークンに分解されます。このトークンを用いて転置インデックスが構築されます。

次に TF/IDF ベースの [BM25](https://ja.wikipedia.org/wiki/Okapi_BM25) スコアリングアルゴリズムによって、文章中からトークンの一致頻度を見て関連性スコアが決定されます。細かくはトークンのレア度や文章中の密度なども重みづけされます。キーワード検索では、わざとスペルミスしたので、**源実朝**という一つのトークンにならずに、**一文字ずつのトークンになって関係ない人名の部分でヒット**してしまったりしてますね。また、「特徴」というワードをそのまま検索しており、文章中に「和歌の特徴」とあからさまに書いていない限りこのワードは意味を成しません。残念ながらこの検索方法では、「和歌の特徴を知りたい」というユーザーの意図は考慮されません。

In [ ]:
from langchain.retrievers import AzureCognitiveSearchRetriever

query = "源実友のお歌にはどのような特徴があったのでしょうか？"
retriever = AzureCognitiveSearchRetriever(
    service_name="gptkb-xxxxxxxxx", #Your Azure AI Search service name
    index_name=index_name,
    api_key=vector_store_password,
    content_key="content",
    top_k=3,
)

docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)
    

# 2. ベクトル類似性検索

# 2.1. シンプルなベクトル検索
「源実**友**の**お歌**にはどのような特徴があったのでしょうか？」というわざとスペルミスを入れたり和歌をお歌と言い換えたりしたクエリーで検索をかけます。`text-embeddings-ada-002` で生成したベクトルを検索すると、キーワードの一致にとらわれずに、テキストの類似性だけを見て検索しています。スペルミスも良い具合に無視されて検索されることが分かります。ほしい答えは「万葉風の歌人」という部分です。

In [ ]:
query = "源実友のお歌にはどのような特徴があったのでしょうか？"  

docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

## 2.1.1. 多言語能力
`text-embeddings-ada-002` の多言語能力を確認してみましょう。日本語のドキュメントを英語で検索します。

In [ ]:
query = "What were the characteristics of Minamoto Sanetomo's poetry?"  

docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

# 2.2. ハイブリッド検索
ハイブリッド検索では、キーワード検索とベクトル検索の両方をクエリーとして使います。ハイブリッド検索ではどのように検索スコアを計算しているかといいますと、まずキーワード検索のスコアには Okapi BM25 アルゴリズムによるスコアを採用し、ベクトル検索ではコサイン類似度をスコアとして用いており、両者異なるスコアです。これを融合するシンプルな計算式として、[Reciprocal Rank Fusion(RRF)](https://learn.microsoft.com/azure/search/hybrid-search-ranking) が採用されています。RRF は両者の文書ランクの逆数の和を取ります。つまり、その文書の順位だけを見て、どちらのランキングでも上位にきていればスコアが高くなる仕組みです。

In [ ]:
query = "源実友は征夷大将軍として知られているだけでなく、ある有名な趣味も持っています。それは何ですか。"  
# Perform a hybrid search
docs = vector_store.similarity_search(
    query=query,
    k=5,
    search_type="hybrid",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)


In [ ]:
query = "源実友は征夷大将軍として知られているだけでなく、ある有名な趣味も持っています。それは何ですか。"  
# similarity_search で search_type="hybrid を指定した場合と同じ
docs = vector_store.hybrid_search(
    query=query,
    k=5
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)

# 2.3 セマンティックハイブリッド検索
セマンティックハイブリッド検索（ハイブリッド検索＋セマンティックランカー）は Azure AI Search 独自の検索機能であり、ハイブリッド検索と検索結果を高い精度で並べ替えるリランカー機能（セマンティックランカー）を組み合わせた高度な検索方法です。リランカーは Microsoft 製の言語モデルである [Turing](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/introducing-multilingual-support-for-semantic-search-on-azure/ba-p/2385110) モデルに基づいています。

<div class="alert alert-block alert-success">
<b>Note:</b> LangChain の Issues <a href="https://github.com/langchain-ai/langchain/issues/15355">#15355</a> が解決するまでは、インデックスに `metadata` フィールドを含める必要があります。</div>

In [ ]:
query = "１３人の合議制に含まれるメンバー一覧"  

docs = vector_store.semantic_hybrid_search(
    query=query,
    search_type="semantic_hybrid",
)

for doc in docs:
    print(doc.metadata["sourcepage"])
    print(doc.metadata["@search.score"])
    print(doc.page_content)